# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [ ]:
%pip install datasets causal_conv1d

# Lab

In [3]:
import VAE, importlib
importlib.reload(VAE)
from VAE import MambaVAE

In [10]:
import torch, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [68]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Monday, Tomorrow is Tuesday."], return_tensors='pt').to('cuda')

In [69]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states

In [62]:
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
# print(res.logits)
tokenizer.batch_decode(res.logits.argmax(-1))

[" =: CannotsupportedOperation:code'string')\n to 'get_\nross_rows__\n integers types typesarray typesarray/ are type are are of for this_validch.data can be passed.\n"]

In [82]:
ssm_last_states.pow(2).sum()/2

tensor(4849.4077, device='cuda:0', grad_fn=<DivBackward0>)

In [4]:
model = MambaVAE()

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/517M [00:00<?, ?B/s]

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [93]:
!python Trainer.py

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.
Some weights of the model checkpoint at state-spaces/mamba-130m-hf were not used when initializing MambaForCausalLM: ['backbone.layers.12.mixer.A_log', 'backbone.layers.12.mixer.D', 'backbone.layers.12.mixer.conv1d.bias', 'backbone.layers.12.mixer.conv1d.weight', 'backbone.layers.12.mixer.dt_proj.bias', 'backbone.layers.12.mixer.dt_proj.weight', 'backbone.layers.12.mixer.in_proj.weight', 'backbone.layers.12.mixer.out_proj.weight', 'backbone.layers.12.mixer.x_proj.weight', 'backbone.layers.12.norm.weight', 'backbone.layers.13.mixer.A_log', 'backbone.layers.13.m